Whenever using ML:

    0) Import, dataset and dataloaders creation, device definition
    1) Design the model (the forward pass), the input size and output size
    2) Construct loss and the optimiser
    3) Training loop
         - Forward pass: Compute prediction [inference]
         - Backward pass: Gradients (auto gradient for last one)
         - Compute loss for accuracy check
         - Update weights
    4) Predict output and calculate accuracy (use torch.no_grad())

Without PyTorch

In [1]:
import numpy as np

# model: y = w*x [ = 2*x ]

# Initialisation
x = np.array([1,2,3,4], dtype=np.float32)
y = np.array([2,4,6,8], dtype=np.float32)
w = 0.0

# Forward pass
def forw_pass(x,w):
    y_hat = w*x
    return y_hat

# Loss MSE
def loss(y, y_hat):
    return ((y_hat - y)**2).mean()

# Gradient
def gradient(x,y,y_hat):
    return np.dot(2*x,y_hat-y).mean()

alpha = .01
num_iteration = 15

for epoch in range(num_iteration):
    y_hat = forw_pass(x,w)
    l = loss(y,y_hat)
    dL_dw = gradient(x,y,y_hat)
    w = w - alpha * dL_dw
    print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w:.5f}")

print(f"\nPrediction after training: f(5) = {forw_pass(5,w):.3f}")
    

Epoch:1  Loss:30.00000000  weight:1.20000
Epoch:2  Loss:4.79999924  weight:1.68000
Epoch:3  Loss:0.76800019  weight:1.87200
Epoch:4  Loss:0.12288000  weight:1.94880
Epoch:5  Loss:0.01966083  weight:1.97952
Epoch:6  Loss:0.00314574  weight:1.99181
Epoch:7  Loss:0.00050331  weight:1.99672
Epoch:8  Loss:0.00008053  weight:1.99869
Epoch:9  Loss:0.00001288  weight:1.99948
Epoch:10  Loss:0.00000206  weight:1.99979
Epoch:11  Loss:0.00000033  weight:1.99992
Epoch:12  Loss:0.00000005  weight:1.99997
Epoch:13  Loss:0.00000001  weight:1.99999
Epoch:14  Loss:0.00000000  weight:1.99999
Epoch:15  Loss:0.00000000  weight:2.00000

Prediction after training: f(5) = 10.000


Using Autograd of PyTorch

In [2]:
import torch

# model: y = w*x = 2*x

# Initialisation
x = torch.tensor([1,2,3,4], dtype=torch.float32)
y = torch.tensor([2,4,6,8], dtype=torch.float32)
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# Forward pass
def forw_pass(x):
    return w*x
    
# Loss MSE
def loss(y, y_hat):
    return ((y_hat - y)**2).mean()

# Gradient calc done by autograd
print(f'Prediction before training: f(5) = {forw_pass(5).item():.3f}')
alpha = .01
num_iteration = 100

for epoch in range(num_iteration):
    y_hat = forw_pass(x)
    
    l = loss(y,y_hat)
    l.backward()
    # print(w.grad)

    with torch.no_grad():
        w -= alpha * w.grad
    w.grad.zero_()
    if epoch%10 == 0:
        print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w:.5f}")    
   

print(f"\nPrediction after training: f(5) = {forw_pass(5):.3f}")

Prediction before training: f(5) = 0.000
Epoch:1  Loss:30.00000000  weight:0.30000
Epoch:11  Loss:1.16278565  weight:1.66531
Epoch:21  Loss:0.04506890  weight:1.93411
Epoch:31  Loss:0.00174685  weight:1.98703
Epoch:41  Loss:0.00006770  weight:1.99745
Epoch:51  Loss:0.00000262  weight:1.99950
Epoch:61  Loss:0.00000010  weight:1.99990
Epoch:71  Loss:0.00000000  weight:1.99998
Epoch:81  Loss:0.00000000  weight:2.00000
Epoch:91  Loss:0.00000000  weight:2.00000

Prediction after training: f(5) = 10.000


Using torch.nn module

In [3]:
import torch
import torch.nn as nn
# model: y = w*x = 2*x

# Initialisation
x = torch.tensor([1,2,3,4], dtype=torch.float32)
y = torch.tensor([2,4,6,8], dtype=torch.float32)
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# Forward pass
def forw_pass(x):
    return w*x
    
alpha = .01
num_iteration = 100

# Loss MSE can be defined by the torch.nn package
loss = nn.MSELoss()
optimiser = torch.optim.SGD([w], lr = alpha)

# Training loop
for epoch in range(num_iteration):
    y_hat = forw_pass(x)
    
    l = loss(y,y_hat)
    l.backward()
    # print(w.grad)

    optimiser.step()
    optimiser.zero_grad()
    
    if epoch%10 == 0:
        print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w:.5f}")    
   

print(f"\nPrediction after training: f(5) = {forw_pass(5):.3f}")
    

Epoch:1  Loss:30.00000000  weight:0.30000
Epoch:11  Loss:1.16278565  weight:1.66531
Epoch:21  Loss:0.04506890  weight:1.93411
Epoch:31  Loss:0.00174685  weight:1.98703
Epoch:41  Loss:0.00006770  weight:1.99745
Epoch:51  Loss:0.00000262  weight:1.99950
Epoch:61  Loss:0.00000010  weight:1.99990
Epoch:71  Loss:0.00000000  weight:1.99998
Epoch:81  Loss:0.00000000  weight:2.00000
Epoch:91  Loss:0.00000000  weight:2.00000

Prediction after training: f(5) = 10.000


Using torch.nn to define model

In [4]:
import numpy as np
import torch
import torch.nn as nn

# Initialisation
X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32)
y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32)
X_test = torch.tensor([5], dtype=torch.float32)

[m, n_features] = X.shape
input_size = n_features
outut_size = n_features

# Defining model
class LinRegression(nn.Module):
    def __init__(self,inp_size, out_size):
        super(LinRegression,self).__init__()
        self.lay1 = nn.Linear(inp_size, out_size)
        
    def forward(self, X):
        return self.lay1(X)

model = LinRegression(input_size, outut_size)

print(f"Prediction before training: f(5) = {model(X_test).item():.4f}\n")

# Defining loss and optimiser
loss_category = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(), lr =0.1)

# Training loop

n_iter = 100
for epoch in range(n_iter):
    y_hat = model(X)
    loss = loss_category(y_hat, y)
    loss.backward()
    
    optimiser.step()
    optimiser.zero_grad()
    
    if epoch%10 == 0:
        [w, b] = model.parameters()
        print(f'Epoch:{epoch+1}   loss:{loss}   w:{w[0].item()}')
        
print(f"\nPrediction after training: f(5) = {model(X_test).item():.4f}")
    


Prediction before training: f(5) = -5.3038

Epoch:1   loss:73.61764526367188   w:3.744051694869995
Epoch:11   loss:0.040669478476047516   w:1.931684136390686
Epoch:21   loss:0.008814368396997452   w:1.9248645305633545
Epoch:31   loss:0.004794783424586058   w:1.9441072940826416
Epoch:41   loss:0.00261065480299294   w:1.9587490558624268
Epoch:51   loss:0.001421449356712401   w:1.9695613384246826
Epoch:61   loss:0.0007739465800113976   w:1.9775396585464478
Epoch:71   loss:0.0004213970387354493   w:1.983426809310913
Epoch:81   loss:0.0002294412988703698   w:1.9877707958221436
Epoch:91   loss:0.00012492624227888882   w:1.9909762144088745

Prediction after training: f(5) = 9.9859


Same thing done over and over 

In [5]:
# Step 0
import torch
import torch.nn as nn
import numpy as np

X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32)
y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32)
X_test = torch.tensor([12], dtype=torch.float32)
print("Correct Ans = 24")
# Step 1
[m, n] = X.shape

class LR(nn.Module):
    def __init__(self, inp_size, out_size):
        super(LR,self).__init__()
        self.lay1 = nn.Linear(inp_size,out_size)
    def forward(self,x):
        return self.lay1(x)
model = LR(n,n)
print(f'Prediction before training: f(x) = {model(X_test).item():.2f}')

# Step 2
loss_category = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(), lr = .1)

# Step 3
n_iter = 10000
for epoch in range(n_iter):
    y_hat = model(X)
    loss = loss_category(y, y_hat)
    loss.backward()
    optimiser.step()
    optimiser.zero_grad()
    
print(f'\nPrediction after training: f(x) = {model(X_test).item():.2f}')


Correct Ans = 24
Prediction before training: f(x) = 4.38

Prediction after training: f(x) = 24.00


In [6]:
# Step 0
import numpy as np
import torch
import torch.nn as nn
X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32)
y = torch.tensor([[10],[20],[30],[40]], dtype=torch.float32)
X_train = torch.tensor([5], dtype=torch.float32)

# Step 1
m, n = X.shape

class LinReg(nn.Module):
    def __init__(self, inp_size, out_size):
        super(LinReg, self).__init__()
        self.lay1 = nn.Linear(inp_size, out_size)
    
    def forward(self, inp):
        out = self.lay1(inp)
        return out
model = LinReg(n,n)

# Step 2
lossCat = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(), lr=.1)
print(f"Before training f(X_train)= {model(X_train).item()}")

# Step 3
n_iter = 1000

for epoch in range(n_iter):
    y_hat = model(X)
    loss = loss_category(y, y_hat)
    loss.backward()
    optimiser.step()
    optimiser.zero_grad()

print(f"After training f(X_train)= {model(X_train).item()}")

Before training f(X_train)= 2.4142417907714844
After training f(X_train)= 50.0


In [7]:
# Step 0
import torch
import torch.nn as nn
X = torch.tensor([[1],[4]], dtype = torch.float32)
y = torch.tensor([[3],[12]], dtype = torch.float32)
X_test = torch.tensor([[8],[3],[4],[100]], dtype = torch.float32)
m, n = X.shape

# Step 1
class Mod1(nn.Module):
    def __init__(self, inp_size, out_size):
        super(Mod1,self).__init__()
        self.lay1 = nn.Linear(inp_size, out_size)
    def forward(self, inp):
        return self.lay1(inp)
model = Mod1(n,n)

# step 2
lossCat = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(), lr =.1)

# Step 3
n_iter = 1000
for epoch in range(n_iter):
    y_hat = model(X)
    loss = lossCat(y_hat, y)
    loss.backward()
    optimiser.step()
    optimiser.zero_grad()
    if epoch%100 == 0:
        w,b = model.parameters()
        # print(w)
        print(f"Epoch {epoch}:  Loss={loss}  w={w.item()}")

print(model(X_test))

Epoch 0:  Loss=59.817222595214844  w=5.0503010749816895
Epoch 100:  Loss=6.65372965613642e-08  w=2.9998559951782227
Epoch 200:  Loss=2.7569058147491887e-12  w=2.9999990463256836
Epoch 300:  Loss=4.831690603168681e-13  w=2.999999761581421
Epoch 400:  Loss=4.831690603168681e-13  w=2.999999761581421
Epoch 500:  Loss=4.831690603168681e-13  w=2.999999761581421
Epoch 600:  Loss=4.831690603168681e-13  w=2.999999761581421
Epoch 700:  Loss=4.831690603168681e-13  w=2.999999761581421
Epoch 800:  Loss=4.831690603168681e-13  w=2.999999761581421
Epoch 900:  Loss=4.831690603168681e-13  w=2.999999761581421
tensor([[ 24.0000],
        [  9.0000],
        [ 12.0000],
        [300.0000]], grad_fn=<AddmmBackward0>)
